# Parser for RateBeer

In this notebook, we are parsing the HTML pages from the RateBeer website.

In [2]:
import os
os.chdir('..')

In [3]:
from classes.parser import *

import pandas as pd
import numpy as np
import datetime
import time
import html
import gzip
import re
import os

data_folder = '../data/'

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
parser = Parser(data_folder)

# Parse the breweries from the places

In [4]:
parser.parse_breweries_from_places()

# Parse the beers from the breweries

In [18]:
parser.parse_brewery_files()

# Parse the beer files for information

In [172]:
parser.parse_beer_files_for_information()

# Parse the beer files for the reviews

In [29]:
parser.parse_beer_files_for_reviews()